## Run Feature Engineering

Create all features for the classification model

In [1]:
import shutil
import pandas as pd
import numpy as np
import pickle
import boto3
import os
import sys
from sagemaker import get_execution_role

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from Grocery_Recommender.Feature_Engineering.create_features import *

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

### Import the customer transaction file

In [2]:
role = get_execution_role()
region = boto3.Session().region_name
bucket = "udacity-machine-learning-capstone-data"
key = "udacity_capstone_data/all_trans.pkl"

In [3]:
s3 = boto3.resource("s3")
all_cust_trans = pickle.loads(s3.Bucket(bucket).Object(key).get()["Body"].read())
all_cust_trans.head()

,SHOP_WEEK,SHOP_DATE,SHOP_WEEKDAY,SHOP_HOUR,QUANTITY,SPEND,PROD_CODE,PROD_CODE_10,PROD_CODE_20,PROD_CODE_30,PROD_CODE_40,CUST_CODE,CUST_PRICE_SENSITIVITY,CUST_LIFESTAGE,BASKET_ID,BASKET_SIZE,BASKET_PRICE_SENSITIVITY,BASKET_TYPE,BASKET_DOMINANT_MISSION,STORE_CODE,STORE_FORMAT,STORE_REGION
0,200607,20060415,7,19,1,0.93,PRD0900033,CL00201,DEP00067,G00021,D00005,CUST0000410727,UM,OT,994100100398294,L,MM,Full Shop,Mixed,STORE00001,LS,E02
1,200607,20060413,5,20,1,1.03,PRD0900097,CL00001,DEP00001,G00001,D00001,CUST0000634693,LA,YF,994100100532898,L,LA,Top Up,Fresh,STORE00001,LS,E02
2,200607,20060416,1,14,1,0.98,PRD0900121,CL00063,DEP00019,G00007,D00002,NaN,NaN,NaN,994100100135562,L,MM,Top Up,Grocery,STORE00001,LS,E02
3,200607,20060415,7,19,1,3.07,PRD0900135,CL00201,DEP00067,G00021,D00005,CUST0000410727,UM,OT,994100100398294,L,MM,Full Shop,Mixed,STORE00001,LS,E02
4,200607,20060415,7,19,1,4.81,PRD0900220,CL00051,DEP00013,G00005,D00002,CUST0000410727,UM,OT,994100100398294,L,MM,Full Shop,Mixed,STORE00001,LS,E02


### Import the train and test sets

Import the train and test sets on which to append the features

In [4]:
s3c = boto3.client("s3")

# Train
key = "train_df.csv"
obj = s3c.get_object(Bucket=bucket, Key=key)
train_df = pd.read_csv(obj["Body"])

# Test
key = "test_df.csv"
obj = s3c.get_object(Bucket=bucket, Key=key)
test_df = pd.read_csv(obj["Body"])

### Instantiate the Feature Engineering class

In [5]:
create_feats = create_features(200716, 200815)

### Restrict the customer file to only those customer and product combinations in the train or test set

In [6]:
filtered_cust_trans = create_feats.filter_cust_trans(all_cust_trans, train_df, test_df)
filtered_cust_trans.head()

,SHOP_WEEK,SHOP_DATE,SHOP_WEEKDAY,SHOP_HOUR,QUANTITY,SPEND,PROD_CODE,PROD_CODE_10,PROD_CODE_20,PROD_CODE_30,PROD_CODE_40,CUST_CODE,CUST_PRICE_SENSITIVITY,CUST_LIFESTAGE,BASKET_ID,BASKET_SIZE,BASKET_PRICE_SENSITIVITY,BASKET_TYPE,BASKET_DOMINANT_MISSION,STORE_CODE,STORE_FORMAT,STORE_REGION
27,200717,20070619,3,16,1,1.01,PRD0903074,CL00045,DEP00011,G00004,D00002,CUST0000634693,LA,YF,994106300582513,L,LA,Top Up,Fresh,STORE01668,LS,N03
28,200719,20070708,1,13,1,1.01,PRD0903074,CL00045,DEP00011,G00004,D00002,CUST0000634693,LA,YF,994106500592857,M,MM,Small Shop,Fresh,STORE01668,LS,N03
29,200720,20070715,1,14,1,1.01,PRD0903074,CL00045,DEP00011,G00004,D00002,CUST0000634693,LA,YF,994106600590082,L,MM,Top Up,Fresh,STORE01668,LS,N03
30,200720,20070715,1,19,1,1.01,PRD0903074,CL00045,DEP00011,G00004,D00002,CUST0000634693,LA,YF,994106600590084,S,LA,Small Shop,Fresh,STORE01668,LS,N03
31,200722,20070723,2,11,1,1.01,PRD0903074,CL00045,DEP00011,G00004,D00002,CUST0000634693,LA,YF,994106800583676,M,LA,Top Up,Fresh,STORE01668,LS,N03


### Get spend, visits, quantity by levels of hierarchy and time bands

Obtain the total spend, visits and quantity by customer and all levels of the hierarchy for various time bands e.g. last 4 weeks, 8 weeks, 13 weeks, 26 weeks and 52 weeks.

NOTE:  The observation period ends in 200815

In [7]:
hierarchy_map = create_feats.get_product_hierarchy_map(filtered_cust_trans)
hierarchy_map.head()

,PROD_CODE,PROD_CODE_10,PROD_CODE_20,PROD_CODE_30,PROD_CODE_40
27,PRD0903074,CL00045,DEP00011,G00004,D00002
56,PRD0900691,CL00023,DEP00008,G00004,D00002
191,PRD0903052,CL00031,DEP00008,G00004,D00002
424,PRD0903269,CL00064,DEP00019,G00007,D00002
487,PRD0903518,CL00069,DEP00020,G00007,D00002


In [8]:
cust_summ_final = create_feats.get_sp_vi_qu(
    filtered_cust_trans,
    hierarchy_map,
    [200716, 200742, 200808, 200815],
    ["52", "26", "8", "1"],
    ["PROD_CODE", "PROD_CODE_10", "PROD_CODE_20", "PROD_CODE_30", "PROD_CODE_40"],
)

cust_summ_final.drop(
    ["PROD_CODE_10", "PROD_CODE_20", "PROD_CODE_30", "PROD_CODE_40"],
    axis=1,
    inplace=True,
)
cust_summ_final.head()

,CUST_CODE,PROD_CODE,SPEND_PROD_CODE_52,QUANTITY_PROD_CODE_52,VISITS_PROD_CODE_52,SPEND_PROD_CODE_26,QUANTITY_PROD_CODE_26,VISITS_PROD_CODE_26,SPEND_PROD_CODE_8,QUANTITY_PROD_CODE_8,VISITS_PROD_CODE_8,SPEND_PROD_CODE_1,QUANTITY_PROD_CODE_1,VISITS_PROD_CODE_1,SPEND_PROD_CODE_10_52,QUANTITY_PROD_CODE_10_52,VISITS_PROD_CODE_10_52,SPEND_PROD_CODE_10_26,QUANTITY_PROD_CODE_10_26,VISITS_PROD_CODE_10_26,SPEND_PROD_CODE_10_8,QUANTITY_PROD_CODE_10_8,VISITS_PROD_CODE_10_8,SPEND_PROD_CODE_10_1,QUANTITY_PROD_CODE_10_1,VISITS_PROD_CODE_10_1,SPEND_PROD_CODE_20_52,QUANTITY_PROD_CODE_20_52,VISITS_PROD_CODE_20_52,SPEND_PROD_CODE_20_26,QUANTITY_PROD_CODE_20_26,VISITS_PROD_CODE_20_26,SPEND_PROD_CODE_20_8,QUANTITY_PROD_CODE_20_8,VISITS_PROD_CODE_20_8,SPEND_PROD_CODE_20_1,QUANTITY_PROD_CODE_20_1,VISITS_PROD_CODE_20_1,SPEND_PROD_CODE_30_52,QUANTITY_PROD_CODE_30_52,VISITS_PROD_CODE_30_52,SPEND_PROD_CODE_30_26,QUANTITY_PROD_CODE_30_26,VISITS_PROD_CODE_30_26,SPEND_PROD_CODE_30_8,QUANTITY_PROD_CODE_30_8,VISITS_PROD_CODE_30_8,SPEND_PROD_CODE_30_1,QUANTITY_PROD_CODE_30_1,VISITS_PROD_CODE_30_1,SPEND_PROD_CODE_40_52,QUANTITY_PROD_CODE_40_52,VISITS_PROD_CODE_40_52,SPEND_PROD_CODE_40_26,QUANTITY_PROD_CODE_40_26,VISITS_PROD_CODE_40_26,SPEND_PROD_CODE_40_8,QUANTITY_PROD_CODE_40_8,VISITS_PROD_CODE_40_8,SPEND_PROD_CODE_40_1,QUANTITY_PROD_CODE_40_1,VISITS_PROD_CODE_40_1
0,CUST0000001052,PRD0902277,1.59,3,3,1.06,2.0,2.0,0.53,1.0,1.0,0.00,0.0,0.0,1.59,3,3,1.06,2.0,2.0,0.53,1.0,1.0,0.00,0.0,0.0,1.59,3,3,1.06,2.0,2.0,0.53,1.0,1.0,0.00,0.0,0.0,1.59,3,3,1.06,2.0,2.0,0.53,1.0,1.0,0.00,0.0,0.0,1.59,3,3,1.06,2.0,2.0,0.53,1.0,1.0,0.00,0.0,0.0
1,CUST0000001052,PRD0904358,1.54,1,1,1.54,1.0,1.0,0.00,0.0,0.0,0.00,0.0,0.0,1.54,1,1,1.54,1.0,1.0,0.00,0.0,0.0,0.00,0.0,0.0,1.54,1,1,1.54,1.0,1.0,0.00,0.0,0.0,0.00,0.0,0.0,1.54,1,1,1.54,1.0,1.0,0.00,0.0,0.0,0.00,0.0,0.0,1.54,1,1,1.54,1.0,1.0,0.00,0.0,0.0,0.00,0.0,0.0
2,CUST0000001392,PRD0901672,0.53,1,1,0.53,1.0,1.0,0.53,1.0,1.0,0.00,0.0,0.0,0.53,1,1,0.53,1.0,1.0,0.53,1.0,1.0,0.00,0.0,0.0,0.53,1,1,0.53,1.0,1.0,0.53,1.0,1.0,0.00,0.0,0.0,0.53,1,1,0.53,1.0,1.0,0.53,1.0,1.0,0.00,0.0,0.0,0.53,1,1,0.53,1.0,1.0,0.53,1.0,1.0,0.00,0.0,0.0
3,CUST0000001437,PRD0903678,0.43,43,10,0.20,20.0,4.0,0.09,9.0,2.0,0.00,0.0,0.0,0.43,43,10,0.20,20.0,4.0,0.09,9.0,2.0,0.00,0.0,0.0,0.43,43,10,0.20,20.0,4.0,0.09,9.0,2.0,0.00,0.0,0.0,0.43,43,10,0.20,20.0,4.0,0.09,9.0,2.0,0.00,0.0,0.0,0.43,43,10,0.20,20.0,4.0,0.09,9.0,2.0,0.00,0.0,0.0
4,CUST0000001480,PRD0901265,1.29,1,1,1.29,1.0,1.0,1.29,1.0,1.0,1.29,1.0,1.0,1.29,1,1,1.29,1.0,1.0,1.29,1.0,1.0,1.29,1.0,1.0,1.29,1,1,1.29,1.0,1.0,1.29,1.0,1.0,1.29,1.0,1.0,1.29,1,1,1.29,1.0,1.0,1.29,1.0,1.0,1.29,1.0,1.0,1.29,1,1,1.29,1.0,1.0,1.29,1.0,1.0,1.29,1.0,1.0


### Get change in spend, visits, quantity by hierarchy and time band

In [9]:
chng_features = create_feats.get_chng_features(
    cust_summ_final,
    ["1", "8", "26", "52"],
    ["PROD_CODE", "PROD_CODE_10", "PROD_CODE_20", "PROD_CODE_30", "PROD_CODE_40"],
)

chng_features.head()

,CUST_CODE,PROD_CODE,CHNG_SPEND_PROD_CODE_1_8,CHNG_QUANTITY_PROD_CODE_1_8,CHNG_VISITS_PROD_CODE_1_8,CHNG_SPEND_PROD_CODE_1_26,CHNG_QUANTITY_PROD_CODE_1_26,CHNG_VISITS_PROD_CODE_1_26,CHNG_SPEND_PROD_CODE_1_52,CHNG_QUANTITY_PROD_CODE_1_52,CHNG_VISITS_PROD_CODE_1_52,CHNG_SPEND_PROD_CODE_8_26,CHNG_QUANTITY_PROD_CODE_8_26,CHNG_VISITS_PROD_CODE_8_26,CHNG_SPEND_PROD_CODE_8_52,CHNG_QUANTITY_PROD_CODE_8_52,CHNG_VISITS_PROD_CODE_8_52,CHNG_SPEND_PROD_CODE_26_52,CHNG_QUANTITY_PROD_CODE_26_52,CHNG_VISITS_PROD_CODE_26_52,CHNG_SPEND_PROD_CODE_10_1_8,CHNG_QUANTITY_PROD_CODE_10_1_8,CHNG_VISITS_PROD_CODE_10_1_8,CHNG_SPEND_PROD_CODE_10_1_26,CHNG_QUANTITY_PROD_CODE_10_1_26,CHNG_VISITS_PROD_CODE_10_1_26,CHNG_SPEND_PROD_CODE_10_1_52,CHNG_QUANTITY_PROD_CODE_10_1_52,CHNG_VISITS_PROD_CODE_10_1_52,CHNG_SPEND_PROD_CODE_10_8_26,CHNG_QUANTITY_PROD_CODE_10_8_26,CHNG_VISITS_PROD_CODE_10_8_26,CHNG_SPEND_PROD_CODE_10_8_52,CHNG_QUANTITY_PROD_CODE_10_8_52,CHNG_VISITS_PROD_CODE_10_8_52,CHNG_SPEND_PROD_CODE_10_26_52,CHNG_QUANTITY_PROD_CODE_10_26_52,CHNG_VISITS_PROD_CODE_10_26_52,CHNG_SPEND_PROD_CODE_20_1_8,CHNG_QUANTITY_PROD_CODE_20_1_8,CHNG_VISITS_PROD_CODE_20_1_8,CHNG_SPEND_PROD_CODE_20_1_26,CHNG_QUANTITY_PROD_CODE_20_1_26,CHNG_VISITS_PROD_CODE_20_1_26,CHNG_SPEND_PROD_CODE_20_1_52,CHNG_QUANTITY_PROD_CODE_20_1_52,CHNG_VISITS_PROD_CODE_20_1_52,CHNG_SPEND_PROD_CODE_20_8_26,CHNG_QUANTITY_PROD_CODE_20_8_26,CHNG_VISITS_PROD_CODE_20_8_26,CHNG_SPEND_PROD_CODE_20_8_52,CHNG_QUANTITY_PROD_CODE_20_8_52,CHNG_VISITS_PROD_CODE_20_8_52,CHNG_SPEND_PROD_CODE_20_26_52,CHNG_QUANTITY_PROD_CODE_20_26_52,CHNG_VISITS_PROD_CODE_20_26_52,CHNG_SPEND_PROD_CODE_30_1_8,CHNG_QUANTITY_PROD_CODE_30_1_8,CHNG_VISITS_PROD_CODE_30_1_8,CHNG_SPEND_PROD_CODE_30_1_26,CHNG_QUANTITY_PROD_CODE_30_1_26,CHNG_VISITS_PROD_CODE_30_1_26,CHNG_SPEND_PROD_CODE_30_1_52,CHNG_QUANTITY_PROD_CODE_30_1_52,CHNG_VISITS_PROD_CODE_30_1_52,CHNG_SPEND_PROD_CODE_30_8_26,CHNG_QUANTITY_PROD_CODE_30_8_26,CHNG_VISITS_PROD_CODE_30_8_26,CHNG_SPEND_PROD_CODE_30_8_52,CHNG_QUANTITY_PROD_CODE_30_8_52,CHNG_VISITS_PROD_CODE_30_8_52,CHNG_SPEND_PROD_CODE_30_26_52,CHNG_QUANTITY_PROD_CODE_30_26_52,CHNG_VISITS_PROD_CODE_30_26_52,CHNG_SPEND_PROD_CODE_40_1_8,CHNG_QUANTITY_PROD_CODE_40_1_8,CHNG_VISITS_PROD_CODE_40_1_8,CHNG_SPEND_PROD_CODE_40_1_26,CHNG_QUANTITY_PROD_CODE_40_1_26,CHNG_VISITS_PROD_CODE_40_1_26,CHNG_SPEND_PROD_CODE_40_1_52,CHNG_QUANTITY_PROD_CODE_40_1_52,CHNG_VISITS_PROD_CODE_40_1_52,CHNG_SPEND_PROD_CODE_40_8_26,CHNG_QUANTITY_PROD_CODE_40_8_26,CHNG_VISITS_PROD_CODE_40_8_26,CHNG_SPEND_PROD_CODE_40_8_52,CHNG_QUANTITY_PROD_CODE_40_8_52,CHNG_VISITS_PROD_CODE_40_8_52,CHNG_SPEND_PROD_CODE_40_26_52,CHNG_QUANTITY_PROD_CODE_40_26_52,CHNG_VISITS_PROD_CODE_40_26_52
0,CUST0000001052,PRD0902277,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.50,0.50,0.5,0.333333,0.333333,0.333333,0.666667,0.666667,0.666667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.50,0.50,0.5,0.333333,0.333333,0.333333,0.666667,0.666667,0.666667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.50,0.50,0.5,0.333333,0.333333,0.333333,0.666667,0.666667,0.666667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.50,0.50,0.5,0.333333,0.333333,0.333333,0.666667,0.666667,0.666667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.50,0.50,0.5,0.333333,0.333333,0.333333,0.666667,0.666667,0.666667
1,CUST0000001052,PRD0904358,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000
2,CUST0000001392,PRD0901672,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.00,1.00,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.00,1.00,1.0,1.000000,1.000000,1.00000

### Get time since last purchased by levels of the hierarchy

In [10]:
time_since = create_feats.time_last_purchased(
    filtered_cust_trans,
    hierarchy_map,
    ["PROD_CODE", "PROD_CODE_10", "PROD_CODE_20", "PROD_CODE_30", "PROD_CODE_40"],
)

time_since.drop(
    ["PROD_CODE_10", "PROD_CODE_20", "PROD_CODE_30", "PROD_CODE_40"],
    axis=1,
    inplace=True,
)
time_since.head()

,CUST_CODE,PROD_CODE,TIME_BTWN_MEDIAN_CUST_PROD_CODE,TIME_BTWN_MEDIAN_OVERALL_PROD_CODE,TIME_BTWN_LAST_PROD_CODE,TIME_BTWN_MEDIAN_CUST_PROD_CODE_10,TIME_BTWN_MEDIAN_OVERALL_PROD_CODE_10,TIME_BTWN_LAST_PROD_CODE_10,TIME_BTWN_MEDIAN_CUST_PROD_CODE_20,TIME_BTWN_MEDIAN_OVERALL_PROD_CODE_20,TIME_BTWN_LAST_PROD_CODE_20,TIME_BTWN_MEDIAN_CUST_PROD_CODE_30,TIME_BTWN_MEDIAN_OVERALL_PROD_CODE_30,TIME_BTWN_LAST_PROD_CODE_30,TIME_BTWN_MEDIAN_CUST_PROD_CODE_40,TIME_BTWN_MEDIAN_OVERALL_PROD_CODE_40,TIME_BTWN_LAST_PROD_CODE_40
0,CUST0000001052,PRD0902277,169.0,8.0,98,169.0,8.0,98,169.0,8.0,98,169.0,6.0,98,169.0,7.0,98
1,CUST0000179746,PRD0902277,68.0,8.0,82,68.0,8.0,82,68.0,8.0,82,68.0,6.0,82,68.0,7.0,82
2,CUST0000509290,PRD0902277,7.5,8.0,9,3.0,8.0,1,3.0,8.0,1,3.0,6.0,7,3.0,7.0,7
3,CUST0000640695,PRD0902277,38.0,8.0,7,17.5,8.0,6,17.5,8.0,6,8.5,6.0,6,8.5,7.0,6
4,CUST0000746359,PRD0902277,12.0,8.0,14,3.0,8.0,1,3.0,8.0,1,2.0,6.0,1,1.0,7.0,1


### Get the ratio of time since last purchased over the median time between purchases

In [11]:
time_since_ratios = create_feats.get_time_since_ratios(
    time_since,
    ["PROD_CODE", "PROD_CODE_10", "PROD_CODE_20", "PROD_CODE_30", "PROD_CODE_40"],
)

time_since_ratios.head()

,CUST_CODE,PROD_CODE,TIME_BTWN_RATIO_CUST_PROD_CODE,TIME_BTWN_RATIO_OVERALL_PROD_CODE,TIME_BTWN_RATIO_CUST_PROD_CODE_10,TIME_BTWN_RATIO_OVERALL_PROD_CODE_10,TIME_BTWN_RATIO_CUST_PROD_CODE_20,TIME_BTWN_RATIO_OVERALL_PROD_CODE_20,TIME_BTWN_RATIO_CUST_PROD_CODE_30,TIME_BTWN_RATIO_OVERALL_PROD_CODE_30,TIME_BTWN_RATIO_CUST_PROD_CODE_40,TIME_BTWN_RATIO_OVERALL_PROD_CODE_40
0,CUST0000001052,PRD0902277,0.579882,12.250,0.579882,12.250,0.579882,12.250,0.579882,16.333333,0.579882,14.000000
1,CUST0000179746,PRD0902277,1.205882,10.250,1.205882,10.250,1.205882,10.250,1.205882,13.666667,1.205882,11.714286
2,CUST0000509290,PRD0902277,1.200000,1.125,0.333333,0.125,0.333333,0.125,2.333333,1.166667,2.333333,1.000000
3,CUST0000640695,PRD0902277,0.184211,0.875,0.342857,0.750,0.342857,0.750,0.705882,1.000000,0.705882,0.857143
4,CUST0000746359,PRD0902277,1.166667,1.750,0.333333,0.125,0.333333,0.125,0.500000,0.166667,1.000000,0.142857


### Get total spend, visits and quantity and proportion of total by basket segmentation

For basket size, basket type, basket price sensitivity, basket dominant mission, shop weekday, hour of shop and store format calculate the key metrics

In [12]:
# Create a shop day part segment
def create_day_part(df):
    if df["SHOP_HOUR"] >= 8 and df["SHOP_HOUR"] <= 11:
        return "MORNING"
    elif df["SHOP_HOUR"] >= 12 and df["SHOP_HOUR"] <= 16:
        return "AFTERNOON"
    elif df["SHOP_HOUR"] >= 17:
        return "EVENING"

In [13]:
filtered_cust_trans.loc[:, "DAY_PART"] = filtered_cust_trans.apply(
    create_day_part, axis=1
)
filtered_cust_trans["DAY_PART"].value_counts()

AFTERNOON    68515
EVENING      42329
MORNING      35456
Name: DAY_PART, dtype: int64

In [14]:
# Create a weekday or weekend segment
def wkday_wkend(df):
    if df["SHOP_WEEKDAY"] == 1 or df["SHOP_WEEKDAY"] == 7:
        return "WEEKEND"
    else:
        return "WEEKDAY"

In [15]:
filtered_cust_trans.loc[:, "WKDAY_WKEND"] = filtered_cust_trans.apply(
    wkday_wkend, axis=1
)
filtered_cust_trans["WKDAY_WKEND"].value_counts()

WEEKDAY    105144
WEEKEND     41156
Name: WKDAY_WKEND, dtype: int64

In [16]:
seg_summary_cust = create_feats.create_seg_summary(
    filtered_cust_trans,
    [
        "BASKET_PRICE_SENSITIVITY",
        "BASKET_SIZE",
        "DAY_PART",
        "WKDAY_WKEND",
        "BASKET_TYPE",
        "BASKET_DOMINANT_MISSION",
        "STORE_FORMAT",
    ],
    "CUST_CODE",
)
seg_summary_cust.head()

,CUST_CODE,BASKET_PRICE_SENSITIVITY_SPEND_CUST_CODE_LA,BASKET_PRICE_SENSITIVITY_SPEND_CUST_CODE_MM,BASKET_PRICE_SENSITIVITY_SPEND_CUST_CODE_UM,BASKET_PRICE_SENSITIVITY_SPEND_CUST_CODE_XX,BASKET_PRICE_SENSITIVITY_QUANTITY_CUST_CODE_LA,BASKET_PRICE_SENSITIVITY_QUANTITY_CUST_CODE_MM,BASKET_PRICE_SENSITIVITY_QUANTITY_CUST_CODE_UM,BASKET_PRICE_SENSITIVITY_QUANTITY_CUST_CODE_XX,BASKET_PRICE_SENSITIVITY_VISITS_CUST_CODE_LA,BASKET_PRICE_SENSITIVITY_VISITS_CUST_CODE_MM,BASKET_PRICE_SENSITIVITY_VISITS_CUST_CODE_UM,BASKET_PRICE_SENSITIVITY_VISITS_CUST_CODE_XX,BASKET_PRICE_SENSITIVITY_PROP_SPEND_CUST_CODE_LA,BASKET_PRICE_SENSITIVITY_PROP_SPEND_CUST_CODE_MM,BASKET_PRICE_SENSITIVITY_PROP_SPEND_CUST_CODE_UM,BASKET_PRICE_SENSITIVITY_PROP_SPEND_CUST_CODE_XX,BASKET_PRICE_SENSITIVITY_PROP_QUANTITY_CUST_CODE_LA,BASKET_PRICE_SENSITIVITY_PROP_QUANTITY_CUST_CODE_MM,BASKET_PRICE_SENSITIVITY_PROP_QUANTITY_CUST_CODE_UM,BASKET_PRICE_SENSITIVITY_PROP_QUANTITY_CUST_CODE_XX,BASKET_PRICE_SENSITIVITY_PROP_VISITS_CUST_CODE_LA,BASKET_PRICE_SENSITIVITY_PROP_VISITS_CUST_CODE_MM,BASKET_PRICE_SENSITIVITY_PROP_VISITS_CUST_CODE_UM,BASKET_PRICE_SENSITIVITY_PROP_VISITS_CUST_CODE_XX,BASKET_SIZE_SPEND_CUST_CODE_L,BASKET_SIZE_SPEND_CUST_CODE_M,BASKET_SIZE_SPEND_CUST_CODE_S,BASKET_SIZE_QUANTITY_CUST_CODE_L,BASKET_SIZE_QUANTITY_CUST_CODE_M,BASKET_SIZE_QUANTITY_CUST_CODE_S,BASKET_SIZE_VISITS_CUST_CODE_L,BASKET_SIZE_VISITS_CUST_CODE_M,BASKET_SIZE_VISITS_CUST_CODE_S,BASKET_SIZE_PROP_SPEND_CUST_CODE_L,BASKET_SIZE_PROP_SPEND_CUST_CODE_M,BASKET_SIZE_PROP_SPEND_CUST_CODE_S,BASKET_SIZE_PROP_QUANTITY_CUST_CODE_L,BASKET_SIZE_PROP_QUANTITY_CUST_CODE_M,BASKET_SIZE_PROP_QUANTITY_CUST_CODE_S,BASKET_SIZE_PROP_VISITS_CUST_CODE_L,BASKET_SIZE_PROP_VISITS_CUST_CODE_M,BASKET_SIZE_PROP_VISITS_CUST_CODE_S,DAY_PART_SPEND_CUST_CODE_AFTERNOON,DAY_PART_SPEND_CUST_CODE_EVENING,DAY_PART_SPEND_CUST_CODE_MORNING,DAY_PART_QUANTITY_CUST_CODE_AFTERNOON,DAY_PART_QUANTITY_CUST_CODE_EVENING,DAY_PART_QUANTITY_CUST_CODE_MORNING,DAY_PART_VISITS_CUST_CODE_AFTERNOON,DAY_PART_VISITS_CUST_CODE_EVENING,DAY_PART_VISITS_CUST_CODE_MORNING,DAY_PART_PROP_SPEND_CUST_CODE_AFTERNOON,DAY_PART_PROP_SPEND_CUST_CODE_EVENING,DAY_PART_PROP_SPEND_CUST_CODE_MORNING,DAY_PART_PROP_QUANTITY_CUST_CODE_AFTERNOON,DAY_PART_PROP_QUANTITY_CUST_CODE_EVENING,DAY_PART_PROP_QUANTITY_CUST_CODE_MORNING,DAY_PART_PROP_VISITS_CUST_CODE_AFTERNOON,DAY_PART_PROP_VISITS_CUST_CODE_EVENING,DAY_PART_PROP_VISITS_CUST_CODE_MORNING,WKDAY_WKEND_SPEND_CUST_CODE_WEEKDAY,WKDAY_WKEND_SPEND_CUST_CODE_WEEKEND,WKDAY_WKEND_QUANTITY_CUST_CODE_WEEKDAY,WKDAY_WKEND_QUANTITY_CUST_CODE_WEEKEND,WKDAY_WKEND_VISITS_CUST_CODE_WEEKDAY,WKDAY_WKEND_VISITS_CUST_CODE_WEEKEND,WKDAY_WKEND_PROP_SPEND_CUST_CODE_WEEKDAY,WKDAY_WKEND_PROP_SPEND_CUST_CODE_WEEKEND,WKDAY_WKEND_PROP_QUANTITY_CUST_CODE_WEEKDAY,WKDAY_WKEND_PROP_QUANTITY_CUST_CODE_WEEKEND,WKDAY_WKEND_PROP_VISITS_CUST_CODE_WEEKDAY,WKDAY_WKEND_PROP_VISITS_CUST_CODE_WEEKEND,BASKET_TYPE_SPEND_CUST_CODE_Full Shop,BASKET_TYPE_SPEND_CUST_CODE_Small Shop,BASKET_TYPE_SPEND_CUST_CODE_Top Up,BASKET_TYPE_SPEND_CUST_CODE_XX,BASKET_TYPE_QUANTITY_CUST_CODE_Full Shop,BASKET_TYPE_QUANTITY_CUST_CODE_Small Shop,BASKET_TYPE_QUANTITY_CUST_CODE_Top Up,BASKET_TYPE_QUANTITY_CUST_CODE_XX,BASKET_TYPE_VISITS_CUST_CODE_Full Shop,BASKET_TYPE_VISITS_CUST_CODE_Small Shop,BASKET_TYPE_VISITS_CUST_CODE_Top Up,BASKET_TYPE_VISITS_CUST_CODE_XX,BASKET_TYPE_PROP_SPEND_CUST_CODE_Full Shop,BASKET_TYPE_PROP_SPEND_CUST_CODE_Small Shop,BASKET_TYPE_PROP_SPEND_CUST_CODE_Top Up,BASKET_TYPE_PROP_SPEND_CUST_CODE_XX,BASKET_TYPE_PROP_QUANTITY_CUST_CODE_Full Shop,BASKET_TYPE_PROP_QUANTITY_CUST_CODE_Small Shop,BASKET_TYPE_PROP_QUANTITY_CUST_CODE_Top Up,BASKET_TYPE_PROP_QUANTITY_CUST_CODE_XX,BASKET_TYPE_PROP_VISITS_CUST_CODE_Full Shop,BASKET_TYPE_PROP_VISITS_CUST_CODE_Small Shop,BASKET_TYPE_PROP_VISITS_CUST_CODE_Top Up,BASKET_TYPE_PROP_VISITS_CUST_CODE_XX,BASKET_DOMINANT_MISSION_SPEND_CUST_CODE_Fresh,BASKET_DOMINANT_MISSION_SPEND_CUST_CODE_Grocery,BASKET_DOMINANT_MISSION_SPEND_CUST_

In [17]:
seg_summary_item = create_feats.create_seg_summary(
    filtered_cust_trans,
    [
        "BASKET_PRICE_SENSITIVITY",
        "BASKET_SIZE",
        "DAY_PART",
        "WKDAY_WKEND",
        "BASKET_TYPE",
        "BASKET_DOMINANT_MISSION",
        "STORE_FORMAT",
    ],
    "PROD_CODE",
)
seg_summary_item.head()

,PROD_CODE,BASKET_PRICE_SENSITIVITY_PROP_SPEND_PROD_CODE_LA,BASKET_PRICE_SENSITIVITY_PROP_SPEND_PROD_CODE_MM,BASKET_PRICE_SENSITIVITY_PROP_SPEND_PROD_CODE_UM,BASKET_PRICE_SENSITIVITY_PROP_SPEND_PROD_CODE_XX,BASKET_PRICE_SENSITIVITY_PROP_QUANTITY_PROD_CODE_LA,BASKET_PRICE_SENSITIVITY_PROP_QUANTITY_PROD_CODE_MM,BASKET_PRICE_SENSITIVITY_PROP_QUANTITY_PROD_CODE_UM,BASKET_PRICE_SENSITIVITY_PROP_QUANTITY_PROD_CODE_XX,BASKET_PRICE_SENSITIVITY_PROP_VISITS_PROD_CODE_LA,BASKET_PRICE_SENSITIVITY_PROP_VISITS_PROD_CODE_MM,BASKET_PRICE_SENSITIVITY_PROP_VISITS_PROD_CODE_UM,BASKET_PRICE_SENSITIVITY_PROP_VISITS_PROD_CODE_XX,BASKET_SIZE_PROP_SPEND_PROD_CODE_L,BASKET_SIZE_PROP_SPEND_PROD_CODE_M,BASKET_SIZE_PROP_SPEND_PROD_CODE_S,BASKET_SIZE_PROP_QUANTITY_PROD_CODE_L,BASKET_SIZE_PROP_QUANTITY_PROD_CODE_M,BASKET_SIZE_PROP_QUANTITY_PROD_CODE_S,BASKET_SIZE_PROP_VISITS_PROD_CODE_L,BASKET_SIZE_PROP_VISITS_PROD_CODE_M,BASKET_SIZE_PROP_VISITS_PROD_CODE_S,DAY_PART_PROP_SPEND_PROD_CODE_AFTERNOON,DAY_PART_PROP_SPEND_PROD_CODE_EVENING,DAY_PART_PROP_SPEND_PROD_CODE_MORNING,DAY_PART_PROP_QUANTITY_PROD_CODE_AFTERNOON,DAY_PART_PROP_QUANTITY_PROD_CODE_EVENING,DAY_PART_PROP_QUANTITY_PROD_CODE_MORNING,DAY_PART_PROP_VISITS_PROD_CODE_AFTERNOON,DAY_PART_PROP_VISITS_PROD_CODE_EVENING,DAY_PART_PROP_VISITS_PROD_CODE_MORNING,WKDAY_WKEND_PROP_SPEND_PROD_CODE_WEEKDAY,WKDAY_WKEND_PROP_SPEND_PROD_CODE_WEEKEND,WKDAY_WKEND_PROP_QUANTITY_PROD_CODE_WEEKDAY,WKDAY_WKEND_PROP_QUANTITY_PROD_CODE_WEEKEND,WKDAY_WKEND_PROP_VISITS_PROD_CODE_WEEKDAY,WKDAY_WKEND_PROP_VISITS_PROD_CODE_WEEKEND,BASKET_TYPE_PROP_SPEND_PROD_CODE_Full Shop,BASKET_TYPE_PROP_SPEND_PROD_CODE_Small Shop,BASKET_TYPE_PROP_SPEND_PROD_CODE_Top Up,BASKET_TYPE_PROP_SPEND_PROD_CODE_XX,BASKET_TYPE_PROP_QUANTITY_PROD_CODE_Full Shop,BASKET_TYPE_PROP_QUANTITY_PROD_CODE_Small Shop,BASKET_TYPE_PROP_QUANTITY_PROD_CODE_Top Up,BASKET_TYPE_PROP_QUANTITY_PROD_CODE_XX,BASKET_TYPE_PROP_VISITS_PROD_CODE_Full Shop,BASKET_TYPE_PROP_VISITS_PROD_CODE_Small Shop,BASKET_TYPE_PROP_VISITS_PROD_CODE_Top Up,BASKET_TYPE_PROP_VISITS_PROD_CODE_XX,BASKET_DOMINANT_MISSION_PROP_SPEND_PROD_CODE_Fresh,BASKET_DOMINANT_MISSION_PROP_SPEND_PROD_CODE_Grocery,BASKET_DOMINANT_MISSION_PROP_SPEND_PROD_CODE_Mixed,BASKET_DOMINANT_MISSION_PROP_SPEND_PROD_CODE_Nonfood,BASKET_DOMINANT_MISSION_PROP_SPEND_PROD_CODE_XX,BASKET_DOMINANT_MISSION_PROP_QUANTITY_PROD_CODE_Fresh,BASKET_DOMINANT_MISSION_PROP_QUANTITY_PROD_CODE_Grocery,BASKET_DOMINANT_MISSION_PROP_QUANTITY_PROD_CODE_Mixed,BASKET_DOMINANT_MISSION_PROP_QUANTITY_PROD_CODE_Nonfood,BASKET_DOMINANT_MISSION_PROP_QUANTITY_PROD_CODE_XX,BASKET_DOMINANT_MISSION_PROP_VISITS_PROD_CODE_Fresh,BASKET_DOMINANT_MISSION_PROP_VISITS_PROD_CODE_Grocery,BASKET_DOMINANT_MISSION_PROP_VISITS_PROD_CODE_Mixed,BASKET_DOMINANT_MISSION_PROP_VISITS_PROD_CODE_Nonfood,BASKET_DOMINANT_MISSION_PROP_VISITS_PROD_CODE_XX,STORE_FORMAT_PROP_SPEND_PROD_CODE_LS,STORE_FORMAT_PROP_SPEND_PROD_CODE_MS,STORE_FORMAT_PROP_SPEND_PROD_CODE_SS,STORE_FORMAT_PROP_SPEND_PROD_CODE_XLS,STORE_FORMAT_PROP_QUANTITY_PROD_CODE_LS,STORE_FORMAT_PROP_QUANTITY_PROD_CODE_MS,STORE_FORMAT_PROP_QUANTITY_PROD_CODE_SS,STORE_FORMAT_PROP_QUANTITY_PROD_CODE_XLS,STORE_FORMAT_PROP_VISITS_PROD_CODE_LS,STORE_FORMAT_PROP_VISITS_PROD_CODE_MS,STORE_FORMAT_PROP_VISITS_PROD_CODE_SS,STORE_FORMAT_PROP_VISITS_PROD_CODE_XLS
0,PRD0900001,0.375000,0.625000,0.000000,0.0,0.375000,0.625000,0.000000,0.0,0.250000,0.750000,0.000000,0.0,0.875000,0.125,0.000000,0.875000,0.125,0.000000,0.750000,0.250000,0.0,0.375000,0.125000,0.500000,0.375000,0.125000,0.500000,0.25000,0.250000,0.500000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.500000,0.125000,0.375000,0.0,0.500000,0.125000,0.375000,0.0,0.500000,0.250000,0.25000,0.0,0.625000,0.00,0.375000,0.0,0.0,0.625000,0.00,0.375000,0.0,0.0,0.75000,0.000000,0.250000,0.0,0.0,0.625,0.000,0.375,0.000,0.625,0.000,0.375,0.000,0.750000,0.00000,0.250000,0.000000
1,PRD0900002,0.416667,0.250000,0.333333,0.0,0.416667,0.250000,0.333333,0.0,0.500000,0.300000,0.200000,0.0,0

### Get ALS user and item factors

In [18]:
s3c = boto3.client("s3")

# Item factors
key = "item_factors.csv"
obj = s3c.get_object(Bucket=bucket, Key=key)
item_factors = pd.read_csv(obj["Body"])

# Rename the columns
cols = [col for col in item_factors if "PROD_CODE" not in col]
cols = ["ITEM_" + col for col in cols]
cols = cols + ["PROD_CODE"]
item_factors.columns = cols

# User factors
key = "user_factors.csv"
obj = s3c.get_object(Bucket=bucket, Key=key)
user_factors = pd.read_csv(obj["Body"])

# Rename the columns
cols = [col for col in user_factors if "CUST_CODE" not in col]
cols = ["USER_" + col for col in cols]
cols = cols + ["CUST_CODE"]
user_factors.columns = cols

### Append the features to the training and test sets and save

In [19]:
train_df_features = create_feats.merge_train_test(
    train_df,
    cust_summ_final,
    chng_features,
    time_since,
    time_since_ratios,
    seg_summary_cust,
    seg_summary_item,
    user_factors,
    item_factors,
)

In [20]:
train_df_features.head()

,CUST_CODE,PROD_CODE,TARGET,SPEND_PROD_CODE_52,QUANTITY_PROD_CODE_52,VISITS_PROD_CODE_52,SPEND_PROD_CODE_26,QUANTITY_PROD_CODE_26,VISITS_PROD_CODE_26,SPEND_PROD_CODE_8,QUANTITY_PROD_CODE_8,VISITS_PROD_CODE_8,SPEND_PROD_CODE_1,QUANTITY_PROD_CODE_1,VISITS_PROD_CODE_1,SPEND_PROD_CODE_10_52,QUANTITY_PROD_CODE_10_52,VISITS_PROD_CODE_10_52,SPEND_PROD_CODE_10_26,QUANTITY_PROD_CODE_10_26,VISITS_PROD_CODE_10_26,SPEND_PROD_CODE_10_8,QUANTITY_PROD_CODE_10_8,VISITS_PROD_CODE_10_8,SPEND_PROD_CODE_10_1,QUANTITY_PROD_CODE_10_1,VISITS_PROD_CODE_10_1,SPEND_PROD_CODE_20_52,QUANTITY_PROD_CODE_20_52,VISITS_PROD_CODE_20_52,SPEND_PROD_CODE_20_26,QUANTITY_PROD_CODE_20_26,VISITS_PROD_CODE_20_26,SPEND_PROD_CODE_20_8,QUANTITY_PROD_CODE_20_8,VISITS_PROD_CODE_20_8,SPEND_PROD_CODE_20_1,QUANTITY_PROD_CODE_20_1,VISITS_PROD_CODE_20_1,SPEND_PROD_CODE_30_52,QUANTITY_PROD_CODE_30_52,VISITS_PROD_CODE_30_52,SPEND_PROD_CODE_30_26,QUANTITY_PROD_CODE_30_26,VISITS_PROD_CODE_30_26,SPEND_PROD_CODE_30_8,QUANTITY_PROD_CODE_30_8,VISITS_PROD_CODE_30_8,SPEND_PROD_CODE_30_1,QUANTITY_PROD_CODE_30_1,VISITS_PROD_CODE_30_1,SPEND_PROD_CODE_40_52,QUANTITY_PROD_CODE_40_52,VISITS_PROD_CODE_40_52,SPEND_PROD_CODE_40_26,QUANTITY_PROD_CODE_40_26,VISITS_PROD_CODE_40_26,SPEND_PROD_CODE_40_8,QUANTITY_PROD_CODE_40_8,VISITS_PROD_CODE_40_8,SPEND_PROD_CODE_40_1,QUANTITY_PROD_CODE_40_1,VISITS_PROD_CODE_40_1,CHNG_SPEND_PROD_CODE_1_8,CHNG_QUANTITY_PROD_CODE_1_8,CHNG_VISITS_PROD_CODE_1_8,CHNG_SPEND_PROD_CODE_1_26,CHNG_QUANTITY_PROD_CODE_1_26,CHNG_VISITS_PROD_CODE_1_26,CHNG_SPEND_PROD_CODE_1_52,CHNG_QUANTITY_PROD_CODE_1_52,CHNG_VISITS_PROD_CODE_1_52,CHNG_SPEND_PROD_CODE_8_26,CHNG_QUANTITY_PROD_CODE_8_26,CHNG_VISITS_PROD_CODE_8_26,CHNG_SPEND_PROD_CODE_8_52,CHNG_QUANTITY_PROD_CODE_8_52,CHNG_VISITS_PROD_CODE_8_52,CHNG_SPEND_PROD_CODE_26_52,CHNG_QUANTITY_PROD_CODE_26_52,CHNG_VISITS_PROD_CODE_26_52,CHNG_SPEND_PROD_CODE_10_1_8,CHNG_QUANTITY_PROD_CODE_10_1_8,CHNG_VISITS_PROD_CODE_10_1_8,CHNG_SPEND_PROD_CODE_10_1_26,CHNG_QUANTITY_PROD_CODE_10_1_26,CHNG_VISITS_PROD_CODE_10_1_26,CHNG_SPEND_PROD_CODE_10_1_52,CHNG_QUANTITY_PROD_CODE_10_1_52,CHNG_VISITS_PROD_CODE_10_1_52,CHNG_SPEND_PROD_CODE_10_8_26,CHNG_QUANTITY_PROD_CODE_10_8_26,CHNG_VISITS_PROD_CODE_10_8_26,CHNG_SPEND_PROD_CODE_10_8_52,CHNG_QUANTITY_PROD_CODE_10_8_52,CHNG_VISITS_PROD_CODE_10_8_52,CHNG_SPEND_PROD_CODE_10_26_52,CHNG_QUANTITY_PROD_CODE_10_26_52,CHNG_VISITS_PROD_CODE_10_26_52,CHNG_SPEND_PROD_CODE_20_1_8,CHNG_QUANTITY_PROD_CODE_20_1_8,CHNG_VISITS_PROD_CODE_20_1_8,CHNG_SPEND_PROD_CODE_20_1_26,CHNG_QUANTITY_PROD_CODE_20_1_26,CHNG_VISITS_PROD_CODE_20_1_26,CHNG_SPEND_PROD_CODE_20_1_52,CHNG_QUANTITY_PROD_CODE_20_1_52,CHNG_VISITS_PROD_CODE_20_1_52,CHNG_SPEND_PROD_CODE_20_8_26,CHNG_QUANTITY_PROD_CODE_20_8_26,CHNG_VISITS_PROD_CODE_20_8_26,CHNG_SPEND_PROD_CODE_20_8_52,CHNG_QUANTITY_PROD_CODE_20_8_52,CHNG_VISITS_PROD_CODE_20_8_52,CHNG_SPEND_PROD_CODE_20_26_52,CHNG_QUANTITY_PROD_CODE_20_26_52,CHNG_VISITS_PROD_CODE_20_26_52,CHNG_SPEND_PROD_CODE_30_1_8,CHNG_QUANTITY_PROD_CODE_30_1_8,CHNG_VISITS_PROD_CODE_30_1_8,CHNG_SPEND_PROD_CODE_30_1_26,CHNG_QUANTITY_PROD_CODE_30_1_26,CHNG_VISITS_PROD_CODE_30_1_26,CHNG_SPEND_PROD_CODE_30_1_52,CHNG_QUANTITY_PROD_CODE_30_1_52,CHNG_VISITS_PROD_CODE_30_1_52,CHNG_SPEND_PROD_CODE_30_8_26,CHNG_QUANTITY_PROD_CODE_30_8_26,CHNG_VISITS_PROD_CODE_30_8_26,CHNG_SPEND_PROD_CODE_30_8_52,CHNG_QUANTITY_PROD_CODE_30_8_52,CHNG_VISITS_PROD_CODE_30_8_52,CHNG_SPEND_PROD_CODE_30_26_52,CHNG_QUANTITY_PROD_CODE_30_26_52,CHNG_VISITS_PROD_CODE_30_26_52,CHNG_SPEND_PROD_CODE_40_1_8,CHNG_QUANTITY_PROD_CODE_40_1_8,CHNG_VISITS_PROD_CODE_40_1_8,CHNG_SPEND_PROD_CODE_40_1_26,CHNG_QUANTITY_PROD_CODE_40_1_26,CHNG_VISITS_PROD_CODE_40_1_26,CHNG_SPEND_PROD_CODE_40_1_52,CHNG_QUANTITY_PROD_CODE_40_1_52,CHNG_VISITS_PROD_CODE_40_1_52,CHNG_SPEND_PROD_CODE_40_8_26,CHNG_QUANTITY_PROD_CODE_40_8_26,CHNG_VISITS_PROD_CODE_40_8_26,CHNG_SPEND_PROD_CODE_40_8_52,CHNG_QUANTITY_PROD_CODE_40_8_52,CHNG_VISITS_PROD_CODE_40_8_52,CHNG_SPEND_PROD_CODE_40_26_52,CHNG_QUANTITY_PROD_CODE_40

In [21]:
test_df_features = create_feats.merge_train_test(
    test_df,
    cust_summ_final,
    chng_features,
    time_since,
    time_since_ratios,
    seg_summary_cust,
    seg_summary_item,
    user_factors,
    item_factors,
)

In [22]:
test_df_features.head()

,CUST_CODE,PROD_CODE,TARGET,SPEND_PROD_CODE_52,QUANTITY_PROD_CODE_52,VISITS_PROD_CODE_52,SPEND_PROD_CODE_26,QUANTITY_PROD_CODE_26,VISITS_PROD_CODE_26,SPEND_PROD_CODE_8,QUANTITY_PROD_CODE_8,VISITS_PROD_CODE_8,SPEND_PROD_CODE_1,QUANTITY_PROD_CODE_1,VISITS_PROD_CODE_1,SPEND_PROD_CODE_10_52,QUANTITY_PROD_CODE_10_52,VISITS_PROD_CODE_10_52,SPEND_PROD_CODE_10_26,QUANTITY_PROD_CODE_10_26,VISITS_PROD_CODE_10_26,SPEND_PROD_CODE_10_8,QUANTITY_PROD_CODE_10_8,VISITS_PROD_CODE_10_8,SPEND_PROD_CODE_10_1,QUANTITY_PROD_CODE_10_1,VISITS_PROD_CODE_10_1,SPEND_PROD_CODE_20_52,QUANTITY_PROD_CODE_20_52,VISITS_PROD_CODE_20_52,SPEND_PROD_CODE_20_26,QUANTITY_PROD_CODE_20_26,VISITS_PROD_CODE_20_26,SPEND_PROD_CODE_20_8,QUANTITY_PROD_CODE_20_8,VISITS_PROD_CODE_20_8,SPEND_PROD_CODE_20_1,QUANTITY_PROD_CODE_20_1,VISITS_PROD_CODE_20_1,SPEND_PROD_CODE_30_52,QUANTITY_PROD_CODE_30_52,VISITS_PROD_CODE_30_52,SPEND_PROD_CODE_30_26,QUANTITY_PROD_CODE_30_26,VISITS_PROD_CODE_30_26,SPEND_PROD_CODE_30_8,QUANTITY_PROD_CODE_30_8,VISITS_PROD_CODE_30_8,SPEND_PROD_CODE_30_1,QUANTITY_PROD_CODE_30_1,VISITS_PROD_CODE_30_1,SPEND_PROD_CODE_40_52,QUANTITY_PROD_CODE_40_52,VISITS_PROD_CODE_40_52,SPEND_PROD_CODE_40_26,QUANTITY_PROD_CODE_40_26,VISITS_PROD_CODE_40_26,SPEND_PROD_CODE_40_8,QUANTITY_PROD_CODE_40_8,VISITS_PROD_CODE_40_8,SPEND_PROD_CODE_40_1,QUANTITY_PROD_CODE_40_1,VISITS_PROD_CODE_40_1,CHNG_SPEND_PROD_CODE_1_8,CHNG_QUANTITY_PROD_CODE_1_8,CHNG_VISITS_PROD_CODE_1_8,CHNG_SPEND_PROD_CODE_1_26,CHNG_QUANTITY_PROD_CODE_1_26,CHNG_VISITS_PROD_CODE_1_26,CHNG_SPEND_PROD_CODE_1_52,CHNG_QUANTITY_PROD_CODE_1_52,CHNG_VISITS_PROD_CODE_1_52,CHNG_SPEND_PROD_CODE_8_26,CHNG_QUANTITY_PROD_CODE_8_26,CHNG_VISITS_PROD_CODE_8_26,CHNG_SPEND_PROD_CODE_8_52,CHNG_QUANTITY_PROD_CODE_8_52,CHNG_VISITS_PROD_CODE_8_52,CHNG_SPEND_PROD_CODE_26_52,CHNG_QUANTITY_PROD_CODE_26_52,CHNG_VISITS_PROD_CODE_26_52,CHNG_SPEND_PROD_CODE_10_1_8,CHNG_QUANTITY_PROD_CODE_10_1_8,CHNG_VISITS_PROD_CODE_10_1_8,CHNG_SPEND_PROD_CODE_10_1_26,CHNG_QUANTITY_PROD_CODE_10_1_26,CHNG_VISITS_PROD_CODE_10_1_26,CHNG_SPEND_PROD_CODE_10_1_52,CHNG_QUANTITY_PROD_CODE_10_1_52,CHNG_VISITS_PROD_CODE_10_1_52,CHNG_SPEND_PROD_CODE_10_8_26,CHNG_QUANTITY_PROD_CODE_10_8_26,CHNG_VISITS_PROD_CODE_10_8_26,CHNG_SPEND_PROD_CODE_10_8_52,CHNG_QUANTITY_PROD_CODE_10_8_52,CHNG_VISITS_PROD_CODE_10_8_52,CHNG_SPEND_PROD_CODE_10_26_52,CHNG_QUANTITY_PROD_CODE_10_26_52,CHNG_VISITS_PROD_CODE_10_26_52,CHNG_SPEND_PROD_CODE_20_1_8,CHNG_QUANTITY_PROD_CODE_20_1_8,CHNG_VISITS_PROD_CODE_20_1_8,CHNG_SPEND_PROD_CODE_20_1_26,CHNG_QUANTITY_PROD_CODE_20_1_26,CHNG_VISITS_PROD_CODE_20_1_26,CHNG_SPEND_PROD_CODE_20_1_52,CHNG_QUANTITY_PROD_CODE_20_1_52,CHNG_VISITS_PROD_CODE_20_1_52,CHNG_SPEND_PROD_CODE_20_8_26,CHNG_QUANTITY_PROD_CODE_20_8_26,CHNG_VISITS_PROD_CODE_20_8_26,CHNG_SPEND_PROD_CODE_20_8_52,CHNG_QUANTITY_PROD_CODE_20_8_52,CHNG_VISITS_PROD_CODE_20_8_52,CHNG_SPEND_PROD_CODE_20_26_52,CHNG_QUANTITY_PROD_CODE_20_26_52,CHNG_VISITS_PROD_CODE_20_26_52,CHNG_SPEND_PROD_CODE_30_1_8,CHNG_QUANTITY_PROD_CODE_30_1_8,CHNG_VISITS_PROD_CODE_30_1_8,CHNG_SPEND_PROD_CODE_30_1_26,CHNG_QUANTITY_PROD_CODE_30_1_26,CHNG_VISITS_PROD_CODE_30_1_26,CHNG_SPEND_PROD_CODE_30_1_52,CHNG_QUANTITY_PROD_CODE_30_1_52,CHNG_VISITS_PROD_CODE_30_1_52,CHNG_SPEND_PROD_CODE_30_8_26,CHNG_QUANTITY_PROD_CODE_30_8_26,CHNG_VISITS_PROD_CODE_30_8_26,CHNG_SPEND_PROD_CODE_30_8_52,CHNG_QUANTITY_PROD_CODE_30_8_52,CHNG_VISITS_PROD_CODE_30_8_52,CHNG_SPEND_PROD_CODE_30_26_52,CHNG_QUANTITY_PROD_CODE_30_26_52,CHNG_VISITS_PROD_CODE_30_26_52,CHNG_SPEND_PROD_CODE_40_1_8,CHNG_QUANTITY_PROD_CODE_40_1_8,CHNG_VISITS_PROD_CODE_40_1_8,CHNG_SPEND_PROD_CODE_40_1_26,CHNG_QUANTITY_PROD_CODE_40_1_26,CHNG_VISITS_PROD_CODE_40_1_26,CHNG_SPEND_PROD_CODE_40_1_52,CHNG_QUANTITY_PROD_CODE_40_1_52,CHNG_VISITS_PROD_CODE_40_1_52,CHNG_SPEND_PROD_CODE_40_8_26,CHNG_QUANTITY_PROD_CODE_40_8_26,CHNG_VISITS_PROD_CODE_40_8_26,CHNG_SPEND_PROD_CODE_40_8_52,CHNG_QUANTITY_PROD_CODE_40_8_52,CHNG_VISITS_PROD_CODE_40_8_52,CHNG_SPEND_PROD_CODE_40_26_52,CHNG_QUANTITY_PROD_CODE_40

### Append the customer segment features

In [23]:
train_df_features = create_feats.append_cust_features(
    filtered_cust_trans, train_df_features
)
test_df_features = create_feats.append_cust_features(
    filtered_cust_trans, test_df_features
)

In [24]:
# Upload the train and test sets to s3
train_df_features.to_csv("train_df_features.csv", index=False)
key = "train_df_features.csv"  # filepath in s3
s3.Bucket(bucket).Object(key).upload_file("train_df_features.csv")

In [25]:
# Upload the train and test sets to s3
test_df_features.to_csv("test_df_features.csv", index=False)
key = "test_df_features.csv"  # filepath in s3
s3.Bucket(bucket).Object(key).upload_file("test_df_features.csv")